# OLSAbsorb: Absorbing categorical variables in OLS

One of the main usecases for this is absorbing fixed effects in panel data.

The fixed effects are included as sparse dummy matrix and partialled out of the main explanatory variables.

In [2]:
import time

import numpy as np
from scipy import sparse
import pandas as pd

from statsmodels.regression.linear_model import OLS
from statsmodels.regression.special_linear_model import OLSAbsorb, cat2dummy_sparse
from statsmodels.tools._sparse import PartialingSparse, dummy_sparse


In [40]:
k_cat1, k_cat2 = 500, 100

keep = (np.random.rand(k_cat1 * k_cat2) > 0.1).astype(bool)

xcat1 = np.repeat(np.arange(k_cat1), k_cat2)[keep]
xcat2 = np.tile(np.arange(k_cat2), k_cat1)[keep]
exog_absorb = np.column_stack((xcat1, xcat2))
nobs = len(xcat1)

exog_sparse = cat2dummy_sparse(exog_absorb)
beta_sparse = 1. / np.r_[np.arange(1, k_cat1), np.arange(1, k_cat2 + 1)]

np.random.seed(999)
beta_dense = np.ones(3)
exog_dense = np.column_stack((np.ones(exog_sparse.shape[0]), np.random.randn(exog_sparse.shape[0], len(beta_dense) - 1)))
y = exog_dense.dot(beta_dense) + exog_sparse.dot(beta_sparse) + 0.01 * np.random.randn(nobs)


In [41]:
exog_absorb.shape, exog_sparse.shape, beta_sparse.shape

((45004, 2), (45004, 599), (599,))

In [42]:
t0 = time.time()
mod_absorb = OLSAbsorb(y, exog_dense, exog_absorb)
res_absorb = mod_absorb.fit()
t1 = time.time()
print('time: ', t1 - t0)

time:  0.027363061904907227


In [43]:
print(res_absorb.summary())

                         OLSAbsorb Regression Results                         
Dep. Variable:                      y   R-squared:                       1.000
Model:                      OLSAbsorb   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.521e+06
Date:                Thu, 21 Jan 2016   Prob (F-statistic):               0.00
Time:                        10:46:17   Log-Likelihood:             1.4387e+05
No. Observations:               45004   AIC:                        -2.865e+05
Df Residuals:                   44403   BIC:                        -2.813e+05
Df Model:                         600                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0653    4.7e-05   2.27e+04      0.0

In [44]:
xcat1

array([  0,   0,   0, ..., 499, 499, 499])

In [45]:
xcat2

array([ 0,  1,  2, ..., 96, 97, 98])

In [46]:
locals().keys()

dict_keys(['__loader__', '_i11', 'pd', '_iii', 'OLSAbsorb', '_i32', '_17', '_36', '_4', '_i44', '_i34', '_38', '_i12', '_i8', '_i30', 't0', 'beta_dense', '_i41', '_34', '_41', '_24', '_i4', '_i1', '_8', 'axis', '_12', '__name__', 'k_cat', '__', '_i29', '_i3', '_27', '_i9', 'exog_sparse', '_16', 'DataFrame', '_i7', '_44', '_i16', '_35', '_i28', '__spec__', '_i43', '_29', 'sparse', '_21', '_i2', '_sh', '_i15', '___', '_i24', '_i39', '_i13', '_10', 'np', 'exog_dense', '_i45', '_', '_i5', '_30', 'df', '_i25', 'res_absorb', '_ih', '_i27', '_i37', '_i42', 'beta_sparse', 'get_ipython', '_i46', '_13', '_i40', '_33', 'mod_absorb', '_45', 'PartialingSparse', '_i19', 'xm', '_i23', 't1', '_i31', '_i14', '_i', '_28', '_19', '_ii', '_i21', 'quit', 'keep', '_i33', '_i38', '_26', '_7', 'cat2dummy_sparse', '_i6', 'y', '_9', 'Out', '_31', 'xcat2', '__package__', 'dummy_sparse', '_i22', '__builtin__', '_dh', '_oh', '_18', '_i10', 'k_cat1', 'nobs', '_11', '_i36', '__doc__', 'xcat1', '_i35', '_20', 'exog_a

In [47]:
exog_sparse.nnz

89918

In [48]:
exog_sparse


<45004x599 sparse matrix of type '<class 'numpy.float64'>'
	with 89918 stored elements in Compressed Sparse Row format>

In [49]:
exog_sparse.T.dot(exog_sparse)

<599x599 sparse matrix of type '<class 'numpy.float64'>'
	with 90427 stored elements in Compressed Sparse Column format>

In [51]:
xcat2.reshape(k_cat1, k_cat2)[:20, :20]

ValueError: total size of new array must be unchanged

In [52]:
xm = exog_dense[:,-1].reshape(k_cat1, k_cat2)

ValueError: total size of new array must be unchanged

In [53]:
xm -= xm.mean(1)[:,None]
xm -= xm.mean(0)

In [54]:
xm[:5, :5]

array([[[ 0.70781951, -0.61444126,  0.13538036, ..., -1.61356043,
         -0.02540206,  1.10654173],
        [-0.6746529 ,  0.33527863,  0.52267286, ..., -0.33361659,
          0.28892705, -0.20139616],
        [-0.49991086,  1.30025533, -0.63031488, ...,  0.57215395,
          0.01590037, -0.38327518],
        [-0.03373329, -2.87631067,  0.64663125, ..., -1.15282776,
          0.07557066, -0.24256912],
        [ 0.47507535,  0.03091861,  0.28423046, ...,  0.09650512,
         -0.06270204, -0.97275079]],

       [[-0.10115587,  0.29664766, -0.18332798, ...,  0.52677218,
         -0.52345954,  0.19811873],
        [ 0.75963979, -1.3796931 ,  0.45493439, ...,  0.25359948,
          0.75988452,  0.16494683],
        [ 0.03308297,  0.93162743,  0.68000504, ...,  0.45301085,
          0.09529476,  0.10493187],
        [-0.69411134,  1.14314916, -1.02699353, ...,  0.5014093 ,
         -1.05200108,  0.67283725],
        [ 0.01058241, -0.47407763, -0.80274294, ..., -0.43139638,
         -0.29

In [17]:
res_absorb.model.exog[:20, -1]

array([ 1.2516452 , -0.98768209, -0.81293526, -2.16324902,  0.86941574,
        0.16520761, -0.73303638,  0.71600731,  1.49542599, -0.92503197,
       -1.05676421, -1.64690497,  2.01773133, -1.93617259, -0.52465453,
        0.28930665, -1.3996646 , -0.10786457,  0.42267147,  1.03173704])

In [18]:
xm.ravel()[:20]

array([ 1.2516452 , -0.98768209, -0.81293526, -2.16324902,  0.86941574,
        0.16520761, -0.73303638,  0.71600731,  1.49542599, -0.92503197,
       -1.05676421, -1.64690497,  2.01773133, -1.93617259, -0.52465453,
        0.28930665, -1.3996646 , -0.10786457,  0.42267147,  1.03173704])

In [21]:
np.max(np.abs(xm.ravel() - res_absorb.model.exog[:, -1]))

0.0

In [20]:
(xm.ravel(), res_absorb.model.exog[:, -1])[:20]

(array([ 1.2516452 ,  0.98768209,  0.81293526, ...,  0.02695404,
         1.25582162,  1.21407205]),
 array([ 1.2516452 ,  0.98768209,  0.81293526, ...,  0.02695404,
         1.25582162,  1.21407205]))

In [26]:
xm = exog_dense.reshape(-1, k_cat1, k_cat2)
xm -= xm.mean(1, keepdims=True)
xm -= xm.mean(2, keepdims=True)
np.max(np.abs(xm.reshape(-1, exog_dense.shape[-1]) - res_absorb.model.exog))

0.0

In [27]:
xm = exog_dense.reshape(-1, k_cat1, k_cat2)
xm -= np.nanmean(xm, axis=1, keepdims=True)
xm -= np.nanmean(xm, axis=2, keepdims=True)
np.max(np.abs(xm.reshape(-1, exog_dense.shape[-1]) - res_absorb.model.exog))

0.0

In [141]:
k_cat = (k_cat1, k_cat2)
xm = exog_dense.reshape(-1, *k_cat)
for axis in range(xm.ndim):
    xm -= np.nanmean(xm, axis=axis, keepdims=True)
np.max(np.abs(xm.reshape(-1, exog_dense.shape[-1]) - res_absorb.model.exog))

ValueError: total size of new array must be unchanged

In [252]:
from pandas import DataFrame
np.random.seed(1234)
df = DataFrame({'A' : np.random.randint(0,10,size=100), 'B' : np.random.randn(100)})
df['C'] = df['B'] - df.groupby('A')['B'].transform('mean')
df.head()
df[df.A==3]

,A,B,C
0,3,0.299347,0.412738
21,3,-0.063758,0.049633
26,3,-1.000889,-0.887498
28,3,0.159520,0.272911
30,3,0.028340,0.141731
32,3,-0.712358,-0.598967
51,3,-0.288721,-0.175330
60,3,0.418900,0.532291
99,3,0.139099,0.252490


In [253]:
df[df.A==3].B.mean()

-0.11339119534010732

In [254]:
df.head()

,A,B,C
0,3,0.299347,0.412738
1,6,0.127277,0.161831
2,5,0.926190,0.838768
3,4,2.455240,1.855498
4,8,-0.320890,0.129203


In [255]:
df['B'] -= df.groupby('A')['B'].transform('mean')
df.head()

,A,B,C
0,3,0.412738,0.412738
1,6,0.161831,0.161831
2,5,0.838768,0.838768
3,4,1.855498,1.855498
4,8,0.129203,0.129203


In [257]:
np.random.seed(1234)
df = DataFrame({'A' : np.random.randint(0,10,size=100), 'B' : np.random.randn(100), 'D' : np.random.randn(100)})
df[['B', 'D']] -= df.groupby('A')[['B', 'D']].transform('mean')
df[df.A==3].B.mean()

-6.1679056923619804e-18

In [258]:
df[df.A==3]

,A,B,D
0,3,0.412738,-0.257500
21,3,0.049633,-0.613769
26,3,-0.887498,-0.414497
28,3,0.272911,1.268343
30,3,0.141731,-0.714718
32,3,-0.598967,0.549872
51,3,-0.175330,-0.013277
60,3,0.532291,1.490721
99,3,0.252490,-1.295175


In [268]:
np.random.seed(1234)
df = DataFrame({'A' : np.random.randint(0,10,size=100), 'B' : np.random.randn(100), 'D' : np.random.randn(100)})
df2 = df[['B', 'D']].copy()
df2[df2.columns] -= df2.groupby(df['A'])[df2.columns].transform('mean')
df2[df.A==3].B.mean()

-6.1679056923619804e-18

In [38]:
import pandas as pd
pd.__version__

'0.17.0'

In [168]:
# with unbalanced panel

k_cat = (k_cat1, k_cat2)
xm = np.empty(exog_dense.shape[1:] + k_cat)
xm.fill(np.nan)
xm[:, xcat1, xcat2] = exog_dense.T
for it in range(3):
    for axis in range(1, xm.ndim):
        xm = xm - np.nanmean(xm, axis=axis, keepdims=True)
np.max(np.abs(xm.reshape(exog_dense.shape[-1], -1).T[keep] + exog_dense.mean(0) - res_absorb.model.wexog), axis=0)

array([  2.46291876e-12,   1.81591915e-08,   8.87074103e-09])

In [161]:
np.mean(np.abs(xm.reshape(exog_dense.shape[-1], -1).T[keep] - res_absorb.model.wexog), axis=0)

array([  1.00000000e+00,   5.50480222e-03,   4.37210890e-04])

In [165]:
exog_dense.mean(0)

array([  1.00000000e+00,   5.50480222e-03,  -4.37210890e-04])

In [143]:
xm.shape

(3, 500, 100)

In [144]:
xm[2, :5, :5].T

array([[ 1.25613714, -0.64928293,         nan,  1.01937572, -0.97799897],
       [-1.05059664, -0.4049158 , -0.12842189,  1.18230988,         nan],
       [-0.77676263, -0.32479844,  0.95401314, -0.25217179, -0.56684707],
       [-2.2978674 , -0.42271699, -0.38449664,  0.12053973, -0.55790378],
       [ 0.71525705,  1.68877901,  0.63852735,  0.26383767, -0.9373546 ]])

In [148]:
xm.reshape(exog_dense.shape[-1], -1).T[:35]

array([[ 0.        ,  0.04164011,  1.25613714],
       [ 0.        ,  0.27399995, -1.05059664],
       [ 0.        , -0.19539316, -0.77676263],
       [ 0.        ,  1.52001694, -2.2978674 ],
       [ 0.        ,  1.37016205,  0.71525705],
       [ 0.        , -0.44608325,  0.18861505],
       [ 0.        , -1.11596406, -0.7916908 ],
       [ 0.        , -0.19718592,  0.61622113],
       [ 0.        , -0.14156295,  1.42509703],
       [ 0.        ,  1.01912515, -0.96210196],
       [ 0.        , -0.07267247, -1.02366338],
       [ 0.        , -0.68477153, -1.70689319],
       [ 0.        ,  1.77287893,  2.07257506],
       [ 0.        ,  0.09987039, -2.00679704],
       [ 0.        , -0.37646237, -0.49738476],
       [ 0.        ,  0.57775695,  0.33491472],
       [ 0.        , -1.71541644, -1.47055459],
       [ 0.        ,  0.33396242, -0.11219462],
       [ 0.        , -0.42031679,  0.30636725],
       [ 0.        ,  1.08465167,  1.03490046],
       [ 0.        ,  0.02006508, -0.044

In [155]:
res_absorb.model.wexog[:5]

array([[ 1.        ,  0.04426225,  1.2568567 ],
       [ 1.        ,  0.27806371, -1.05106664],
       [ 1.        , -0.18939071, -0.77632501],
       [ 1.        ,  1.52568043, -2.29819365],
       [ 1.        ,  1.37558971,  0.71402554]])

In [135]:
(1 - np.isnan(xm)).sum(2)

array([[90, 89, 90, ..., 89, 95, 92],
       [90, 89, 90, ..., 89, 95, 92],
       [90, 89, 90, ..., 89, 95, 92]])

In [93]:
keep[:25]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [55]:
exog_dense.shape + k_cat

(45004, 3, 500, 100)

In [59]:
xm.shape

(3, 500, 100)

In [60]:
xm[:, xcat1, xcat2].shape

(3, 45004)

In [63]:
xm.shape

(3, 500, 100)

In [64]:
xm[-1, :5,:5]

array([[ 0.82354472, -0.79173107, -0.4527107 , -2.03858391,  0.02011735],
       [-0.22716557, -0.62813354, -0.55430332, -0.31087111,  0.59286123],
       [        nan, -0.17556253,  0.62983789,  0.27106117,  1.27671329],
       [ 0.32628804,  0.82303316,  0.24584554,  0.23520047,  0.57318511],
       [-0.11265914,         nan, -0.56664384,  0.3148938 , -1.02569297]])

In [66]:
xm.reshape(-1, exog_dense.shape[-1])[:15]


array([[-0.43259242,  0.25886556,  0.32405193],
       [ 0.25928348, -0.6951397 ,  0.08582273],
       [ 0.63588495, -0.1396784 , -0.4278447 ],
       [-0.01900773,  0.36544528,  0.79722157],
       [-1.28181799,  0.63564222,  0.29128238],
       [-0.30422389,  1.06199035, -0.0739226 ],
       [ 0.03798318, -0.70651738,  0.00825467],
       [-0.44490113,  0.43204658, -0.26864732],
       [-0.04345138, -0.14047757,  0.10359401],
       [-0.3030682 , -0.03983098, -0.21627441],
       [        nan, -0.86480285, -0.06625034],
       [ 0.60778669,  0.03822536, -0.73526058],
       [ 0.08891526, -0.83772098, -0.15563802],
       [-0.35842707, -0.20229687,         nan],
       [ 0.57887614, -0.40193665,  0.05807339]])

In [67]:
keep[:15]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True], dtype=bool)

In [68]:
res_absorb.model.exog[:15]

array([[ 1.        ,  0.12715784,  1.40189088],
       [ 1.        ,  0.31481499, -0.85844916],
       [ 1.        , -0.26613444, -0.64890071],
       [ 1.        ,  1.56626757, -2.09137019],
       [ 1.        ,  1.45632806,  0.94529342],
       [ 1.        , -0.40020119,  0.3152273 ],
       [ 1.        , -1.11006083, -0.58482153],
       [ 1.        , -0.18840956,  0.81302365],
       [ 1.        , -0.16130472,  1.60087155],
       [ 1.        ,  0.98434258, -0.83544737],
       [ 1.        , -0.18664934, -0.85806707],
       [ 1.        , -0.75977816, -1.51205424],
       [ 1.        ,  1.74777474,  2.13005498],
       [ 1.        ,  0.07115422, -1.84164224],
       [ 1.        , -0.3525907 , -0.40890379]])

In [69]:
(1-keep).sum()

4996

In [70]:
keep.shape

(50000,)

In [130]:
k_cat = (k_cat1, k_cat2)
xm = np.empty(exog_dense.shape[1:] + k_cat)
xm.fill(np.nan)
xm2 = xm.copy()
xm[:, xcat1, xcat2] = exog_dense.T
xm2[2, xcat1, xcat2] = exog_dense[:, 2]

In [131]:
xm[:, :5, :5]

array([[[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
        [        nan,  1.        ,  1.        ,  1.        ,  1.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
        [ 1.        ,         nan,  1.        ,  1.        ,  1.        ]],

       [[ 0.12715784,  0.31481499, -0.26613444,  1.56626757,  1.45632806],
        [-0.57838256,  1.06855293,  0.89574066,  0.08659882,  1.63830921],
        [        nan,  0.23843596, -0.12445156, -1.60814904, -2.53914201],
        [ 1.2175538 ,  0.00848407, -1.24047272, -0.34612256, -1.0335652 ],
        [-1.54299242,         nan,  0.48500646, -2.02472795,  1.27804609]],

       [[ 1.40189088, -0.85844916, -0.64890071, -2.09137019,  0.94529342],
        [-0.79131045, -0.50054959, -0.48471778, -0.50400105,  1.63103412],
        [        nan, -0.23832739,  0.77982209, -0.48005241,  0.56651075],
        [ 1.06236213,

In [114]:
np.nonzero(np.isnan(xm[2, :, :].ravel()))[0][:5]

array([30, 41, 47, 54, 65], dtype=int64)

In [74]:
exog_dense[:15, 2]

array([ 1.40189088, -0.85844916, -0.64890071, -2.09137019,  0.94529342,
        0.3152273 , -0.58482153,  0.81302365,  1.60087155, -0.83544737,
       -0.85806707, -1.51205424,  2.13005498, -1.84164224, -0.40890379])

In [76]:
exog_dense[500:515, 2]

array([ 1.06431957, -0.45038494,  0.15887889,  1.73184946,  0.67221779,
        1.30466212,  0.10767882, -0.7530067 ,  0.22635772,  0.46991989,
       -0.26436464,  1.16655405,  1.36775412,  0.13920489,  1.42370617])

In [80]:
exog_dense[xcat1 == 2][:5]

array([[ 1.        ,  0.23843596, -0.23832739],
       [ 1.        , -0.12445156,  0.77982209],
       [ 1.        , -1.60814904, -0.48005241],
       [ 1.        , -2.53914201,  0.56651075],
       [ 1.        , -2.47332647, -2.6726669 ]])

In [83]:
np.nanmean(xm, axis=axis, keepdims=True)[:10]

array([[[  1.81260902e-17],
        [ -3.88699527e-18],
        [  7.75701677e-18],
        [ -3.98286287e-18],
        [  1.55479811e-17],
        [ -1.97812566e-18],
        [ -9.04254047e-18],
        [ -1.99143143e-18],
        [ -1.98697633e-18],
        [  7.89491929e-18],
        [ -1.14850658e-17],
        [ -8.01966970e-18],
        [ -2.13317961e-17],
        [  3.94745964e-18],
        [  0.00000000e+00],
        [ -1.93925419e-18],
        [  1.01148302e-17],
        [  1.19218580e-17],
        [  2.01401002e-17],
        [ -9.31312912e-18],
        [ -1.59672525e-17],
        [ -1.00700501e-17],
        [  0.00000000e+00],
        [  1.91831192e-18],
        [  1.29733926e-17],
        [ -1.98697633e-18],
        [  2.90888129e-18],
        [  1.11996182e-17],
        [  1.18952467e-17],
        [  4.08357894e-18],
        [ -3.94745964e-18],
        [ -9.80329382e-18],
        [  1.97372982e-18],
        [  6.86230567e-18],
        [  1.18952467e-17],
        [  3.9918131

In [95]:
xcat2[:20]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [96]:
np.nonzero(1 - keep)[0][:5]

array([30, 41, 47, 54, 65], dtype=int64)

In [104]:
exog_absorb[:35]

array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0,  8],
       [ 0,  9],
       [ 0, 10],
       [ 0, 11],
       [ 0, 12],
       [ 0, 13],
       [ 0, 14],
       [ 0, 15],
       [ 0, 16],
       [ 0, 17],
       [ 0, 18],
       [ 0, 19],
       [ 0, 20],
       [ 0, 21],
       [ 0, 22],
       [ 0, 23],
       [ 0, 24],
       [ 0, 25],
       [ 0, 26],
       [ 0, 27],
       [ 0, 28],
       [ 0, 29],
       [ 0, 31],
       [ 0, 32],
       [ 0, 33],
       [ 0, 34],
       [ 0, 35]])

In [119]:
xm[2, :5, :5]

array([[ 1.40189088, -0.85844916, -0.64890071, -2.09137019,  0.94529342],
       [-0.79131045, -0.50054959, -0.48471778, -0.50400105,  1.63103412],
       [        nan, -0.23832739,  0.77982209, -0.48005241,  0.56651075],
       [ 1.06236213,  1.27169002, -0.22707719,  0.22426961,  0.39110672],
       [-0.89344399,         nan, -0.50018391, -0.41260533, -0.76851699]])

In [124]:
np.nanmean(xm2[2, :,:], axis=0, keepdims=True)[:, :5]

array([[-0.01076965,  0.03562408, -0.02866147,  0.04997381,  0.07351298]])

In [126]:
exog_dense[xcat2 == 2, 2].mean()

-0.028661471528007357

In [105]:
k_cat


(500, 100)

In [115]:
np.isnan(xm[2, :, :].ravel()[keep]).any()

False

In [117]:
exog_dense.shape

(45004, 3)

In [129]:
(xm2[2, :,:] - np.nanmean(xm2[2, :,:], axis=0, keepdims=True)).shape

(500, 100)

In [239]:
def _group_demean_iterative(exog_dense, groups, add_mean=True, max_iter=10, atol=1e-8, get_groupmeans=False):
    """iteratively demean an array for two-way fixed effects
    
    This is intended for almost balanced panels. The data is converted
    to a 3-dimensional array with nans for missing cells.
    
    currently works only for two-way effects
    groups have to be integers corresponding to range(k_cati)
    
    no input error checking
    
    This function will change as more options and special cases are
    included.
    
    Parameters
    ----------
    exog_dense : 2d ndarray
        data with observations in rows and variables in columns.
        This array will currently not be modified.
    groups : 2d ndarray, int
        groups labels specified as consecutive integers starting at zero
    max_iter : int
        maximum number of iterations
    atol : float
        tolerance for convergence. Convergence is achieved if the
        maximum absolute change (np.ptp) is smaller than atol.
        
    Returns
    -------
    ex_dm_w : ndarray
        group demeaned exog_dense array in wide format
    ex_dm : ndarray
        group demeaned exog_dense array in long format
    it : int
        number of iterations used. If convergence has not been
        achieved then it will be equal to max_iter - 1
    
    """
    # with unbalanced panel

    k_cat = tuple((groups.max(0) + 1).tolist())
    xm = np.empty(exog_dense.shape[1:] + k_cat)
    xm.fill(np.nan)
    xm[:, groups[:, 0], groups[:, 1]] = exog_dense.T
    # for final group means
    gmean = []
    if get_groupmeans:
        gmean = [np.nanmean(xm, axis=axis) for axis in range(len(k_cat))]
    keep = ~np.isnan(xm[0]).ravel()
    finished = False
    for it in range(max_iter):
        for axis in range(1, xm.ndim):
            group_mean = np.nanmean(xm, axis=axis, keepdims=True)
            xm -= group_mean
            if np.ptp(group_mean) < atol:
                finished = True
                break
        if finished:
            break
    
    xd = xm.reshape(exog_dense.shape[-1], -1).T[keep]
    if add_mean:
        xmean = exog_dense.mean(0)
        xd += xmean
        xm += xmean[:, None, None]
    return xm, xd, it

xm, xd, it = _group_demean_iterative(exog_dense, exog_absorb, max_iter=50, add_mean=False)
xm.shape, it

((3, 500, 100), 3)

In [240]:
np.max(np.abs(xm.reshape(exog_dense.shape[-1], -1).T[keep] + exog_dense.mean(0) - res_absorb.model.wexog), axis=0)

array([  2.46291876e-12,   3.38842843e-11,   1.62518887e-11])

In [212]:
np.max(np.abs(xd + exog_dense.mean(0) - res_absorb.model.wexog), axis=0)

array([  2.46291876e-12,   3.38842843e-11,   1.62518887e-11])

In [213]:
xm, xd, it = _group_demean_iterative(exog_dense, exog_absorb, max_iter=50, add_mean=True)
np.max(np.abs(xd - res_absorb.model.wexog), axis=0)

array([  2.46291876e-12,   3.38842843e-11,   1.62518887e-11])

In [214]:
xd.shape

(45004, 3)

In [227]:
ym, yd, it = _group_demean_iterative(y[:,None], exog_absorb, max_iter=50, add_mean=True)

In [228]:
mod_ols2 = OLS(yd, xd)
ddof = k_cat1 + k_cat2 - 2
mod_ols2.df_resid = mod_ols2.df_resid - ddof
mod_ols2.df_model = mod_ols2.df_model + ddof
res_ols2 = mod_ols2.fit()

In [229]:
res_ols2.params


array([ 1.06526335,  1.00000761,  0.99998616])

In [230]:
res_absorb.params

array([ 1.06526335,  1.00000761,  0.99998616])

In [231]:
res_ols2.bse

array([  4.69574069e-05,   4.71519470e-05,   4.75109106e-05])

In [232]:
res_absorb.bse

array([  4.69574069e-05,   4.71519470e-05,   4.75109106e-05])

In [233]:
res_ols2.bse / res_absorb.bse

array([ 1.,  1.,  1.])

In [234]:
ddof

598

In [238]:
res_ols2.df_resid, res_absorb.df_resid, res_ols2.df_model, res_absorb.df_model

(44403.0, 44403.0, 600.0, 600.0)

In [236]:
print(res_ols2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.488e+06
Date:                Thu, 21 Jan 2016   Prob (F-statistic):               0.00
Time:                        16:07:46   Log-Likelihood:             1.4387e+05
No. Observations:               45004   AIC:                        -2.865e+05
Df Residuals:                   44403   BIC:                        -2.813e+05
Df Model:                         600                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0653    4.7e-05   2.27e+04      0.0

In [237]:
print(res_absorb.summary())

                         OLSAbsorb Regression Results                         
Dep. Variable:                      y   R-squared:                       1.000
Model:                      OLSAbsorb   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.521e+06
Date:                Thu, 21 Jan 2016   Prob (F-statistic):               0.00
Time:                        16:08:01   Log-Likelihood:             1.4387e+05
No. Observations:               45004   AIC:                        -2.865e+05
Df Residuals:                   44403   BIC:                        -2.813e+05
Df Model:                         600                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0653    4.7e-05   2.27e+04      0.0

In [ ]:
xx = np.array((1e4, 10))
dfxx = pd.DataFrame(xx)
dfxx.values.base is xx.base